In [29]:
import numpy as np
import pandas as pd

from nltk import word_tokenize, sent_tokenize
import nltk
from collections import Counter
stop_words = nltk.corpus.stopwords.words('english')

from urllib import request
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
tromso_description = """Tromsø is a beautiful city between FJORDS, ISLANDS AND MOUNTAINS, with a visible past, a fascinating history, a lively, colourful city centre, an inclusive nightlife and numerous attractions. Use the city as a base to foray into Arctic wilderness chasing Midnight Sun and Northern Lights. 01. 02."""
oslo_description = """Oslo is considered as a global city and is the major Norwegian hub for trading, shipping and banking. Location of Oslo: OSLO IS POSITIONED AT THE NORTHERNMOST END OF THE OSLOFJORD and occupies around 40 big and small islands within its limits. The climate of the region is temperate, humid."""
mining_course_description = """The aim of the course is to introduce the students to the concepts and techniques of natural languages processing and analysis, unstructured information analysis and management for better decision- making by deriving valuable insights from enterprise content regardless of source or format. The course provides deep and rich knowledge of text analysis techniques and applications including sentiment analysis and opinion mining, information access and text mining, document classification, topic extraction and other techniques and applications using real-world data and cases."""

In [26]:
def token_pro(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens] # everything lowercase
    tokens = [word.strip() for word in tokens] # strips whitespaces
    tokens = [word for word in tokens if word.isalnum()] # only alpha numerical values
    tokens = [word for word in tokens if not word in stop_words] # removes stopwords
    return tokens 

def genBow(tokens):
    bow = Counter(tokens)
    return bow

def printBow(bow, queryString):
    query_frequency = Counter(queryString)
    tf_vec = []
    for word in bow:
        if word in queryString:
            tf = query_frequency[word] #/len(bow)
        else:
            tf = 0
        tf_vec.append(tf)
    return tf_vec

In [23]:
token_tromso = token_pro(tromso_description)
token_oslo = token_pro(oslo_description)
token_nlp =  token_pro(mining_course_description)


print(token_tromso[:10])
print(token_oslo[:10])
print(token_nlp[:10])


['tromsø', 'beautiful', 'city', 'fjords', 'islands', 'mountains', 'visible', 'past', 'fascinating', 'history']
['oslo', 'considered', 'global', 'city', 'major', 'norwegian', 'hub', 'trading', 'shipping', 'banking']
['aim', 'course', 'introduce', 'students', 'concepts', 'techniques', 'natural', 'languages', 'processing', 'analysis']


In [16]:
url = "https://www.gutenberg.org/files/2554/2554-0.txt"
response = request.urlopen(url)
raw = response.read().decode('utf8')
print(raw[:200])

token_CP = token_pro(raw)
print(token_CP[:10])

The Project Gutenberg eBook of Crime and Punishment, by Fyodor Dostoevsky

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with a
['project', 'gutenberg', 'ebook', 'crime', 'punishment', 'fyodor', 'dostoevsky', 'ebook', 'use', 'anyone']


In [24]:
corpus = [tromso_description, oslo_description, mining_course_description, raw]
tokens_corpus = token_pro((' ').join(corpus))
tokens_corpus[:10]

['tromsø',
 'beautiful',
 'city',
 'fjords',
 'islands',
 'mountains',
 'visible',
 'past',
 'fascinating',
 'history']

In [28]:
bow_corpus = genBow(tokens_corpus)
bow_corpus.most_common(10)

[('raskolnikov', 782),
 ('one', 638),
 ('would', 572),
 ('know', 524),
 ('said', 518),
 ('could', 496),
 ('come', 476),
 ('man', 474),
 ('like', 452),
 ('though', 443)]

In [30]:
bow_tromso = genBow(token_tromso)
bow_oslo = genBow(token_oslo)
bow_nlp = genBow(token_nlp)
bow_CP = genBow(token_CP)

#running our sentences through the tf function:
tf_tromso = printBow(bow_corpus, bow_tromso)
tf_oslo = printBow(bow_corpus, bow_oslo)
tf_nlp = printBow(bow_corpus, bow_nlp)
tf_CP = printBow(bow_corpus, bow_CP)

#Converting to dataframe for visualization
tf_df= pd.DataFrame([tf_tromso, tf_oslo, tf_nlp, tf_CP])
tf_df

,0,1,2,3,4,5,6,7,8,9,...,9334,9335,9336,9337,9338,9339,9340,9341,9342,9343
0,1,1,3,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,7,3,0,4,0,1,34,5,6,...,1,1,1,1,1,1,1,1,1,1


In [31]:
from sklearn.metrics.pairwise import cosine_similarity
df2 = pd.DataFrame(cosine_similarity(tf_df, dense_output=True))
df2.head()

,0,1,2,3
0,1.000000,0.111154,0.0000,0.008419
1,0.111154,1.000000,0.0000,0.010073
2,0.000000,0.000000,1.0000,0.026400
3,0.008419,0.010073,0.0264,1.000000


In [32]:
d12 = sum(np.asarray(tf_tromso)- np.asarray(tf_oslo))
print(d12)
d13 = sum(np.asarray(tf_tromso)- np.asarray(tf_nlp))
print(d13)

2
-20
